run on WSL-Kali, 

env experiment1-env
<code>source /home/kali/experiment1-env/bin/activate</code>

workingspace <code>cd /mnt/e/Onedrive/1-Documents/1-PhD/2-Experiment/feature_extract</code>

data <code>/mnt/f/My Drive/UNSW_pcap-isolated</code>

In [1]:
import nfstream
print(nfstream.__version__)

6.3.5


In [2]:
import pandas as pd
import glob, os
import numpy as np

In [4]:
from nfstream import NFStreamer, NFPlugin
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [61]:
#source="/mnt/f/My Drive/UNSW_pcap-isolated/Amazon Echo/1.pcap"
source = "/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/42.pcap"

In [60]:
df = NFStreamer(source).to_pandas()
df.head()

AttributeError: 'NoneType' object has no attribute 'head'

# 1-Create CSV

In [64]:
#os.path.getsize (source)
#glob.glob(source)
a=pcap_filepath("/mnt/f/My Drive/UNSW_pcap-isolated","Iphone")
a

['/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/1.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/4.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/12.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/36.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/54.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/55.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/56.pcap',
 '/mnt/f/My Drive/UNSW_pcap-isolated/Iphone/60.pcap']

In [63]:
def pcap_filepath(path,device_label):
    file_path=path+'/'+device_label+'/*.pcap'
    all_files = glob.glob(file_path)
    files_in=[]
    for files in all_files:
        if os.path.getsize(files)>1800:
            #print(files)
            files_in.append(files)
        #else:
            #print(files, 'is empty')
    return files_in

In [6]:
# list all device in dataset from folder name
def get_device_list(path):
    p = os.listdir(path)
    device_list=[]
    for i in p:
        x = path+'/'+i
        if os.path.isdir(x):
            device_list.append(i)
            #print(x)
    return device_list

In [7]:
def feature_extraction(df,device_name, label):
    #df1 = pd.DataFrame()
    df1= df.copy()
    df1['device'] = device_name
    df1['label'] = label
    
    df1.reset_index(drop = True, inplace = True)
    columns_name = df1.columns
    dflist = df1.values.tolist() #convert df to list
    return dflist, columns_name

In [12]:
#feature extract, label and combine 
def data_preparation(path,device_list,label):
    mylist = []
    i=0
    j=0
    while i < len(device_list):
        filepath = pcap_filepath(path,device_list[i])
        for filename in filepath:
            print(filename)
            #load csv
            #df = pd.read_csv(filename, delimiter = ',', dtype=datatype)
            df = NFStreamer(filename).to_pandas()
            # set label and device
            dl, columns_name = feature_extraction(df,device_list[i],label)
            
            # combine
            mylist.extend(dl)
            j+=1
        
        print('combined ',j,' ', device_list[i],' pcap files')
        i+=1
    print('all ',i,' devices combined')
    #change list to dataframe
    mydf = pd.DataFrame(mylist, columns=columns_name)
    return mydf

In [ ]:
# main method

path = "/mnt/f/My Drive/UNSW_pcap-isolated"

#list all device
device_list = get_device_list(path)

#list non iot device
non_iot_dev =['Android Phone 1','Android Phone 2','IPhone','Laptop','MacBook','Samsung Galaxy Tab']

#list iot device
iot_dev = list(set(device_list) - set(non_iot_dev))

#extract feature
df_non_iot = data_preparation(path,non_iot_dev,0)
#df_iot = data_preparation(path,iot_dev,1)

#combine df
#df = df_iot.append(df_non_iot)

#save combined df to csv
#df.to_csv('dataset.csv', index=False, header=False)
df_non_iot.to_csv('non_iot_dataset.csv',index=False)

/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 1/7.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 1/15.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 1/47.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 1/53.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 1/55.pcap
combined  5   Android Phone 1  pcap files
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/1.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/12.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/13.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/15.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/26.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/27.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/28.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/29.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/33.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/19.pcap
/mnt/f/My Drive/UNSW_pcap-isolated/Android Phone 2/20.pcap
/mnt/f/My Drive/

In [109]:
df_non_iot.to_csv('noniot_dataset.csv',index=False)

In [ ]:
filename = 'noniot_dataset.csv'
df1 = pd.read_csv(filename, delimiter = ',')
df1.head(5)

In [118]:
df1.shape

(621840, 40)

In [15]:
df["application_category_name"].unique()

array(['System', 'Network', 'Web'], dtype=object)

In [10]:
df.columns

Index(['id', 'expiration_id', 'src_ip', 'src_mac', 'src_oui', 'src_port',
       'dst_ip', 'dst_mac', 'dst_oui', 'dst_port', 'protocol', 'ip_version',
       'vlan_id', 'tunnel_id', 'bidirectional_first_seen_ms',
       'bidirectional_last_seen_ms', 'bidirectional_duration_ms',
       'bidirectional_packets', 'bidirectional_bytes', 'src2dst_first_seen_ms',
       'src2dst_last_seen_ms', 'src2dst_duration_ms', 'src2dst_packets',
       'src2dst_bytes', 'dst2src_first_seen_ms', 'dst2src_last_seen_ms',
       'dst2src_duration_ms', 'dst2src_packets', 'dst2src_bytes',
       'application_name', 'application_category_name',
       'application_is_guessed', 'requested_server_name', 'client_fingerprint',
       'server_fingerprint', 'user_agent', 'content_type'],
      dtype='object')

In [18]:
from nfstream import NFPlugin, NFStreamer
from sklearn.ensemble import RandomForestClassifier

df = NFStreamer(source).to_pandas()
X = df[["bidirectional_packets", "bidirectional_bytes"]]
y = df["application_category_name"].apply(lambda x: 1 if 'System' in x else 0)
model = RandomForestClassifier()
model.fit(X, y)

RandomForestClassifier()

In [64]:
import numpy
source="/mnt/f/My Drive/UNSW_pcap-isolated/Amazon Echo/20.pcap"
class ModelPrediction(NFPlugin):
    def on_init(self, packet, flow):
        flow.udps.model_prediction = 0
      
    def on_expire(self, flow):
         
        to_predict = numpy.array([flow.bidirectional_packets,
                                  flow.bidirectional_bytes]).reshape((1,-1))
       
        flow.udps.model_prediction = self.my_model.predict(to_predict)
        
ml_streamer = NFStreamer(source, udps=ModelPrediction(my_model=model))
Y=lambda x: 1 if 'System' in x else 0
#test_X=[]
test_y=[]
y_pred=[]
for flow in ml_streamer:
    y=Y(flow.application_category_name)
    #test_X.append(flow.to_predict)
    test_y.append(y)
    y_pred.append(flow.udps.model_prediction)
    print(flow.udps.model_prediction,y)

[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[1] 

[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[0] 0
[0] 0
[0] 0
[0] 0
[0] 0
[1] 1
[1] 1
[1] 1
[1] 1
[0] 0
[0] 

In [71]:
y

0